In [11]:
import numpy as np
import xarray as xr
from importlib import reload

from analysis_package import plotting_functions
from analysis_package import open_datasets
from analysis_package import derive_potential_density_values_TEST
from analysis_package import ecco_masks
from analysis_package import potential_density_overturning

ecco_masks = reload(ecco_masks)
plotting_functions = reload(plotting_functions)
open_datasets = reload(open_datasets)
derive_potential_density_values_TEST = reload(derive_potential_density_values_TEST)
potential_density_overturning = reload(potential_density_overturning)



In [2]:
data_dir = "./nctiles_monthly/"

# PHIHYD: insitu pressure anomaly with respect to the depth integral of gravity and reference density (g*rho_reference)
PHIHYD_var_str = "PHIHYD"
# SALT: insitu salinity (psu)
SALT_var_str = "SALT"
# THETA: potential pressure (C)
THETA_var_str = "THETA"

grid_path = "./ecco_grid/ECCOv4r3_grid.nc"
grid = xr.open_dataset(grid_path)

In [4]:
all_times_times_slice = np.arange(0,288)

PHIHYD_ds_raw = open_datasets.open_combine_raw_ECCO_tile_files(data_dir, PHIHYD_var_str, all_times_times_slice)
SALT_ds_raw = open_datasets.open_combine_raw_ECCO_tile_files(data_dir, SALT_var_str, all_times_times_slice)
THETA_ds_raw = open_datasets.open_combine_raw_ECCO_tile_files(data_dir, THETA_var_str, all_times_times_slice)

PHIHYD_ds_raw = PHIHYD_ds_raw.assign_coords(i=np.arange(0,90),j=np.arange(0,90),k=np.arange(0,50))
SALT_ds_raw = SALT_ds_raw.assign_coords(i=np.arange(0,90),j=np.arange(0,90),k=np.arange(0,50))
THETA_ds_raw = THETA_ds_raw.assign_coords(i=np.arange(0,90),j=np.arange(0,90),k=np.arange(0,50))


Loaded PHIHYD over time slice  

Loaded SALT over time slice  

Loaded THETA over time slice  



In [14]:
print(PHIHYD_ds_raw.dims)

Frozen(SortedKeysDict({'time': 288, 'k': 50, 'j': 90, 'i': 90, 'tile': 13}))


In [13]:
for i in range(1,14):
    PDENS_ds, P_INSITY_ds = derive_potential_density_values_TEST.make_potential_density_dataset(PHIHYD_ds_raw.isel(tile=i-1), 
                                                                                            SALT_ds_raw.isel(tile=i-1), 
                                                                                            THETA_ds_raw.isel(tile=i-1), 
                                                                                            all_times_times_slice, 
                                                                                            ref_pressure=2000.)
    if i < 10:
        PDENS_ds.to_netcdf("./nctiles_monthly/PDENS/PDENS.000"+str(i)+".nc")
    else:
        PDENS_ds.to_netcdf("./nctiles_monthly/PDENS/PDENS.00"+str(i)+".nc")

    print("finished "+str(i))

finished 1
finished 2
finished 3
finished 4
finished 5
finished 6
finished 7
finished 8
finished 9
finished 10
finished 11
finished 12
finished 13


In [15]:
print(PDENS_ds)

<xarray.Dataset>
Dimensions:  (i: 90, j: 90, k: 50, time: 288)
Coordinates:
  * time     (time) float64 1.0 2.0 3.0 4.0 5.0 ... 285.0 286.0 287.0 288.0
  * k        (k) int64 0 1 2 3 4 5 6 7 8 9 10 ... 40 41 42 43 44 45 46 47 48 49
  * j        (j) int64 0 1 2 3 4 5 6 7 8 9 10 ... 80 81 82 83 84 85 86 87 88 89
  * i        (i) int64 0 1 2 3 4 5 6 7 8 9 10 ... 80 81 82 83 84 85 86 87 88 89
    dep      (k) float64 5.0 15.0 25.0 35.0 ... 5.039e+03 5.461e+03 5.906e+03
    tim      (time) datetime64[ns] 1992-01-16 1992-02-16 ... 2015-12-16
    lon      (j, i) float64 -127.5 -127.5 -127.5 -127.5 ... -99.42 -105.6 -111.9
    lat      (j, i) float64 -57.27 -57.8 -58.32 -58.84 ... -88.03 -88.08 -88.1
Data variables:
    PDENS    (time, k, j, i) float64 1.036e+03 1.036e+03 1.036e+03 ... nan nan
    timstep  (time) float64 732.0 1.428e+03 2.172e+03 ... 2.096e+05 2.104e+05
    land     (k, j, i) float64 1.0 1.0 1.0 1.0 1.0 1.0 ... nan nan nan nan nan
    area     (j, i) float64 3.544e+09 3.463e+0

In [16]:
print(SALT_ds_raw)

<xarray.Dataset>
Dimensions:  (i: 90, j: 90, k: 50, tile: 13, time: 288)
Coordinates:
  * time     (time) float64 1.0 2.0 3.0 4.0 5.0 ... 285.0 286.0 287.0 288.0
  * k        (k) int64 0 1 2 3 4 5 6 7 8 9 10 ... 40 41 42 43 44 45 46 47 48 49
  * j        (j) int64 0 1 2 3 4 5 6 7 8 9 10 ... 80 81 82 83 84 85 86 87 88 89
  * i        (i) int64 0 1 2 3 4 5 6 7 8 9 10 ... 80 81 82 83 84 85 86 87 88 89
    dep      (k) float64 5.0 15.0 25.0 35.0 ... 5.039e+03 5.461e+03 5.906e+03
    tim      (time) datetime64[ns] 1992-01-16 1992-02-16 ... 2015-12-16
    lon      (tile, j, i) float64 -111.6 -111.3 -110.9 ... -99.42 -105.6 -111.9
    lat      (tile, j, i) float64 -88.24 -88.38 -88.52 ... -88.03 -88.08 -88.1
Dimensions without coordinates: tile
Data variables:
    SALT     (tile, time, k, j, i) float64 nan nan nan nan ... nan nan nan nan
    timstep  (tile, time) float64 732.0 1.428e+03 ... 2.096e+05 2.104e+05
    land     (tile, k, j, i) float64 nan nan nan nan nan ... nan nan nan nan nan
  